In [13]:
import pandas as pd
import seaborn
import math
from scipy import optimize
import numpy as np
import os
from IPython import embed
from utils.analysis import plot_roc_curve
from utils.analysis import flip_underdog_wins
import utils.glitko as glitko
from sklearn.metrics import log_loss


1. Load cleaned Tennis data 


In [21]:
matches_df = pd.read_csv("../../Data/tennis-data.co.uk/Clean data 2001 - 2016.csv",index_col=0)

matches_df.head()

,Player 1,Player 2,Outcome,set_num,Date,match_id,winner_prob,loser_prob,first_to,Surface
0,Clement A.,Gaudenzi A.,0.0,1,2001-01-01,0,NaN,NaN,2,Hard
1,Clement A.,Gaudenzi A.,1.0,2,2001-01-01,0,NaN,NaN,2,Hard
2,Clement A.,Gaudenzi A.,1.0,3,2001-01-01,0,NaN,NaN,2,Hard
3,Goldstein P.,Jones A.,1.0,1,2001-01-01,1,NaN,NaN,2,Hard
4,Goldstein P.,Jones A.,1.0,2,2001-01-01,1,NaN,NaN,2,Hard


0.6217275680758094

In [23]:


#Other cleaning and sorting
matches_df['Date'] = pd.to_datetime(matches_df['Date'])

#Looking at match level data. Therefore only taking 1 observation per match, and first one (with starting ELOs)
matches_df = matches_df[matches_df['set_num']==1]

matches_df.drop(['loser_prob','set_num','first_to','Surface'],axis=1,inplace=True)

#Winner of ovearll match is always player 1
matches_df['Outcome'] = 1

#Duplicate df
opp_df = matches_df.copy()

opp_df['Outcome'] = 0
opp_df.rename(index=str,columns = {"Player 1":"Player 2","Player 2":"Player 1"},inplace=True)

matches_df = matches_df.append(opp_df,ignore_index=True,sort=True)

matches_df.reset_index(inplace=True,drop=True)

matches_df['Year-Month'] = matches_df['Date'].dt.to_period('M')



In [24]:
ratings_df = pd.DataFrame(index=matches_df['Player 1'].unique())

ratings_df['rat'] = 0

ratings_df['RD'] = 350/173.7178

ratings_df['vol'] = 0.06

In [25]:
for i, period in enumerate(matches_df['Year-Month'].unique()):
    print(period)
    period_df = matches_df[matches_df['Year-Month']==period]
    updated_ratings, period_predictions = glitko.update_glickos(period_df, ratings_df)
    ratings_df.update(updated_ratings,overwrite=True)
    if i == 1:
        matches_df = pd.merge(matches_df,period_predictions,how='left',left_index=True,right_index=True)
    else:
        matches_df.update(period_predictions,overwrite=True)



2001-01
2001-02
2001-03
2001-04
2001-05
2001-06
2001-07
2001-08
2001-09
2001-10
2001-11
2001-12
2002-01
2002-02
2002-03
2002-04
2002-05
2002-06
2002-07
2002-08
2002-09
2002-10
2002-11
2002-12
2003-01
2003-02
2003-03
2003-04
2003-05
2003-06
2003-07
2003-08
2003-09
2003-10
2003-11
2004-01
2004-02
2004-03
2004-04
2004-05
2004-06
2004-07
2004-08
2004-09
2004-10
2004-11
2005-01
2005-02
2005-03
2005-04
2005-05
2005-06
2005-07
2005-08
2005-09
2005-10
2005-11
2006-01
2006-02
2006-03
2006-04
2006-05
2006-06
2006-07
2006-08
2006-09
2006-10
2006-11
2007-01
2007-02
2007-03
2007-04
2007-05
2007-06
2007-07
2007-08
2007-09
2007-10
2007-11
2007-12
2008-01
2008-02
2008-03
2008-04
2008-05
2008-06
2008-07
2008-08
2008-09
2008-10
2008-11
2009-01
2009-02
2009-03
2009-04
2009-05
2009-06
2009-07
2009-08
2009-09
2009-10
2009-11
2010-01
2010-02
2010-03
2010-04
2010-05
2010-06
2010-07
2010-08
2010-09
2010-10
2010-11
2011-01
2011-02
2011-03
2011-04
2011-05
2011-06
2011-07
2011-08
2011-09
2011-10
2011-11
2012-01


In [29]:
not_null_df = matches_df[matches_df['winner_prob'].notnull() & not_null_df['E 1'].notnull()]
log_loss(not_null_df['Outcome'],not_null_df['winner_prob'])

0.8548857727562313

In [30]:
log_loss(not_null_df['Outcome'],not_null_df['E 1'])

0.6523588507366461

In [ ]:
not_null_df